<h1 align="center" style="background-color:#616161;color:white">Outlier Analysis</h1>

<h3>Summary</h3>

* Two types of analysis was conducted
    - Daily LIstening Habits: Analysis of the number of unique tracks vs. num. of plays a user seen on a daily basis
    - Histogram of the time period in between song plays

<h3 style="background-color:#616161;color:white">0. Code setup</h3>

In [65]:
import os
import sys
import pandas as pd
import csv
import json
import sqlite3
from bokeh.charts import *
from bokeh.sampledata.autompg import autompg as df
from bokeh.models import HoverTool, BoxSelectTool,BoxZoomTool,ResetTool
from bokeh.models import NumeralTickFormatter


## Parameters you can change

# Abs path to settings file
root = "C:/DS/Github/MusicRecommendation"  # BA, Windows


## Finish setting up
os.chdir(root)
## Add the prelim module
fPath = root + "/1_codebase"
if fPath not in sys.path: sys.path.append(fPath)
from codebase import *

settingsDict =  loadSettings()

# Load data from database
dbPath = root + settingsDict['dbPath']

def getRandomUsers(maxUsers):
    db = sqlite3.connect(dbPath)
    SQStr ="SELECT userID FROM tblUser ORDER BY RANDOM() LIMIT " + maxUsers

In [74]:
# Add hover to this comma-separated string and see what changes
hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
            ("userID", "@user"),
            ("PlayedTimestamp","@PlayedTimestamp"),
        ]
    )
TOOLS = [BoxSelectTool(),BoxZoomTool(), ResetTool(),hover]

<h3 style="background-color:#616161;color:white">1. Daily listening habits</h3>

<b>Summary</b>
* User 21 listens to a high number of unique songs in a day (up to 460)
* User 8 listens to around 10-50 songs a day, upto 2300 times a day

In [76]:
db = sqlite3.connect(dbPath)

sqlStr ='SELECT Cast(substr(userID,-5) as integer) as user,date(PlayedTimestamp) as PlayedTimeStamp ,count(*) as NumOfPlays, count(Distinct trackID) as NumOfTracks from tblInputData group by userID, date(PlayedTimestamp) ORDER BY NumOfPlays'
res = pd.read_sql_query(sqlStr, db)
db.close()

p = Scatter(res, x='NumOfPlays', y='NumOfTracks', color='user', legend=False, title="Daily listening habits of users", xlabel="NumOfPlays", ylabel="NumOfTracks",tools=TOOLS)
output_notebook()
show(p)

ValueError: tool BoxSelectTool(id='a75e43b5-a090-4e6e-8b0f-7d073fe7ff33', ...) to be added already has 'plot' attribute set

<h3 style="background-color:#616161;color:white">2. Interval time</h3>

In [68]:
reset_output() 
db = sqlite3.connect(dbPath)
sqlStr ='Select userID, (SecsSincePrevPlay/60) as MinsSinceFirstPlay from tblMain order by userID, historyID'
res = pd.read_sql_query(sqlStr, db)
db.close()

In [73]:
usrRes = res.loc[res['MinsSinceFirstPlay'] <= 90]
p = Histogram(usrRes['MinsSinceFirstPlay'], title="Count of intervals <= 90 minutes",bins=30)
output_notebook()

p.yaxis.formatter=NumeralTickFormatter(format="0,0")
p.xaxis.formatter=NumeralTickFormatter(format="0,0")
show(p)

Loading BokehJS ...

In [ ]:
usrRes = res.loc[res['MinsSinceFirstPlay'] >= 90]
p = Histogram(usrRes['MinsSinceFirstPlay'], title="Count of intervals > 30 days", bins=50)
output_notebook()

p.yaxis.formatter=NumeralTickFormatter(format="0,0")
p.xaxis.formatter=NumeralTickFormatter(format="0,0")
show(p)

<h3 style="background-color:#616161;color:white">3. User History</h3>

In [78]:
# Select user ID
usr = 8

reset_output() 
db = sqlite3.connect(dbPath)
sqlStr ='select (SecsSincePrevPlay/60) as MinsSincePrev, historyID, TrackID from tblMain where userID = ' + str(usr)
res = pd.read_sql_query(sqlStr, db)
db.close()


In [ ]:
db = sqlite3.connect(dbPath)

p = Scatter(res, x='historyID', y='MinsSincePrev', color='TrackID', legend=False, title="", xlabel="History", ylabel="NumOfTracks")
output_notebook()
show(p)

<h3 style="background-color:#616161;color:white">END</h3>